<a href="https://colab.research.google.com/github/GHC-av/DL_project/blob/main/DenseNet_WeightsTraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning - Final Project
## DenseNet Weights Training

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import cv2
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import seaborn as sns
import keras
from keras.models import Sequential
from keras import layers
from keras import models
import sklearn
from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix
#from keras.preprocessing.image import array_to_img
from keras.applications import DenseNet121
from keras.preprocessing.image import ImageDataGenerator
#from keras.preprocessing import image_dataset_from_directory
from keras.applications import DenseNet121

In [ ]:
%matplotlib inline
import pandas as pd

In [ ]:
os.listdir("/content/drive/My Drive/CoronaHack")

['Chest_xray_Corona_Metadata.csv',
 'Chest_xray_Corona_dataset_Summary.csv',
 'archive.zip',
 'Coronahack-Chest-XRay-Dataset',
 'weight_14nov.h5',
 'modelDN_15nov.h5',
 'modelCNN_15nov.h5',
 'modelCNN_15novV2.h5',
 'modelDN_15novV2.h5']

# Data set for transfer learning

https://www.kaggle.com/praveengovi/coronahack-chest-xraydataset




In [ ]:
#!unzip -uq "/content/drive/My Drive/CoronaHack/archive.zip" -d "/content/drive/My Drive/CoronaHack"

In [ ]:
train_df = pd.read_csv("/content/drive/My Drive/CoronaHack/Chest_xray_Corona_Metadata.csv")
train_df.shape

(5910, 6)

In [ ]:
train_df.dropna(how = 'all')
train_df.isnull().sum()

Unnamed: 0                   0
X_ray_image_name             0
Label                        0
Dataset_type                 0
Label_2_Virus_category    5841
Label_1_Virus_category    1576
dtype: int64

In [ ]:
train_df.fillna('unknown', inplace=True)
train_df.isnull().sum()

Unnamed: 0                0
X_ray_image_name          0
Label                     0
Dataset_type              0
Label_2_Virus_category    0
Label_1_Virus_category    0
dtype: int64

In [ ]:
train = train_df[train_df['Dataset_type'] == 'TRAIN']
test = train_df[train_df['Dataset_type'] == 'TEST']

#assert train_data.shape[0] + test_data.shape[0] == train_df.shape[0]

print(f"Shape of train data : {train.shape}")
print(f"Shape of test data : {test.shape}")


Shape of train data : (5286, 6)
Shape of test data : (624, 6)


In [ ]:
train.sample(10)

,Unnamed: 0,X_ray_image_name,Label,Dataset_type,Label_2_Virus_category,Label_1_Virus_category
5161,5161,person921_virus_1578.jpeg,Pnemonia,TRAIN,unknown,Virus
1643,1643,person1264_bacteria_3222.jpeg,Pnemonia,TRAIN,unknown,bacteria
4005,4005,person495_virus_1001.jpeg,Pnemonia,TRAIN,unknown,Virus
5257,5271,5e6dd879fde9502400e58b2f.jpeg,Pnemonia,TRAIN,COVID-19,Virus
1747,1747,person1214_virus_2059.jpeg,Pnemonia,TRAIN,unknown,Virus
1211,1211,NORMAL2-IM-1260-0001.jpeg,Normal,TRAIN,unknown,unknown
2254,2254,person1366_bacteria_3490.jpeg,Pnemonia,TRAIN,unknown,bacteria
510,510,IM-0644-0001.jpeg,Normal,TRAIN,unknown,unknown
4387,4387,person542_bacteria_2276.jpeg,Pnemonia,TRAIN,unknown,bacteria
4304,4304,person563_bacteria_2338.jpeg,Pnemonia,TRAIN,unknown,bacteria


In [ ]:
test.sample(10)

,Unnamed: 0,X_ray_image_name,Label,Dataset_type,Label_2_Virus_category,Label_1_Virus_category
5329,5352,IM-0102-0001.jpeg,Normal,TEST,unknown,unknown
5320,5343,NORMAL2-IM-0013-0001.jpeg,Normal,TEST,unknown,unknown
5814,5837,person43_virus_92.jpeg,Pnemonia,TEST,unknown,Virus
5439,5462,NORMAL2-IM-0237-0001.jpeg,Normal,TEST,unknown,unknown
5425,5448,NORMAL2-IM-0273-0001.jpeg,Normal,TEST,unknown,unknown
5494,5517,NORMAL2-IM-0370-0001.jpeg,Normal,TEST,unknown,unknown
5823,5846,person38_virus_83.jpeg,Pnemonia,TEST,unknown,Virus
5433,5456,NORMAL2-IM-0249-0001.jpeg,Normal,TEST,unknown,unknown
5908,5931,person1633_virus_2829.jpeg,Pnemonia,TEST,unknown,Virus
5867,5890,person1680_virus_2897.jpeg,Pnemonia,TEST,unknown,Virus


In [ ]:
train_pn = train.loc[train.Label == 'Pnemonia', 'Label'].count()
train_n = train.loc[train.Label == 'Normal', 'Label'].count()

print(f"there are {train_pn} pneumonia cases in the train data set")
print(f"there are {train_n} normal cases in the train data set")

test_pn = test.loc[test.Label == 'Pnemonia', 'Label'].count()
test_n = test.loc[test.Label == 'Normal', 'Label'].count()

print(f"there are {test_pn} pneumonia cases in the test data set")
print(f"there are {test_n} normal cases in the test data set")



there are 3944 pneumonia cases in the train data set
there are 1342 normal cases in the train data set
there are 390 pneumonia cases in the test data set
there are 234 normal cases in the test data set


In [ ]:
METRICS = [
    'accuracy',
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall')
    ]

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True
                                   )

val_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
train_path = "/content/drive/My Drive/CoronaHack/Coronahack-Chest-XRay-Dataset/train"
test_path = "/content/drive/My Drive/CoronaHack/Coronahack-Chest-XRay-Dataset/test"

In [ ]:
img_size = 150
batch_size = 32

train_set = train_datagen.flow_from_dataframe(
    dataframe=train,
    directory=train_path,
    x_col="X_ray_image_name",
    y_col="Label",
    target_size=(img_size, img_size),
    color_mode="rgb",
    class_mode="binary",
    batch_size= batch_size,
    seed=25,
    shuffle=True
)

val_set = val_datagen.flow_from_dataframe(
    dataframe=test,
    directory=test_path,
    x_col="X_ray_image_name",
    y_col="Label",
    target_size=(img_size, img_size),
    color_mode="rgb",
    class_mode="binary",
    batch_size= batch_size,
    seed=25,
    shuffle=True
)

Found 5286 validated image filenames belonging to 2 classes.
Found 624 validated image filenames belonging to 2 classes.


In [ ]:
print(train_set.class_indices)
print(val_set.class_indices)

{'Normal': 0, 'Pnemonia': 1}
{'Normal': 0, 'Pnemonia': 1}


In [ ]:
densenet = DenseNet121(
    #weights="imagenet",
    include_top=False,
    input_shape=(img_size,img_size,3),
    classes = 2
)

def build_model():
    model = Sequential()
    model.add(densenet)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dropout(0.6))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(
        optimizer=keras.optimizers.Adam(lr=0.0005),
        loss='binary_crossentropy',
        metrics=METRICS
        )
    return model

29089792/29084464 [==============================] - 0s 0us/step


In [ ]:
premodel_DN = build_model()
premodel_DN.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Functional)     (None, 4, 4, 1024)        7037504   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1024)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 1025      
Total params: 7,038,529
Trainable params: 6,954,881
Non-trainable params: 83,648
_________________________________________________________________


In [ ]:
weight_pn = (1 / train_pn)*(train_pn + train_n)/2.0
weight_n = (1 / train_n)*(train_pn + train_n)/2.0

# Order of the classes, mapped to the label indices, is alphanumeric
class_weight = {0: weight_n, 1: weight_pn}

print('Weight for class pneumonia: {:.2f}'.format(weight_pn)) 
print('Weight for class normal: {:.2f}'.format(weight_n))

Weight for class pneumonia: 0.67
Weight for class normal: 1.97


In [ ]:
train_count = train.shape[0]
test_count = test.shape[0]

In [ ]:
history = premodel_DN.fit(train_set,
                    #steps_per_epoch = train_count // batch_size,
                    epochs = 50,
                    validation_data = val_set,
                    class_weight = class_weight,
                    #validation_steps = test_count // batch_size
                    )

premodel_DN.save_weights('/content/drive/My Drive/CoronaHack/weightDN.h5')

Epoch 1/50
166/166 [==============================] - 1186s 7s/step - loss: 0.2405 - accuracy: 0.9077 - precision: 0.9737 - recall: 0.9006 - val_loss: 1.1333 - val_accuracy: 0.8782 - val_precision: 0.8568 - val_recall: 0.9667
Epoch 2/50
166/166 [==============================] - 91s 550ms/step - loss: 0.1234 - accuracy: 0.9529 - precision: 0.9838 - recall: 0.9526 - val_loss: 1.0074 - val_accuracy: 0.7147 - val_precision: 0.6866 - val_recall: 1.0000
Epoch 3/50
166/166 [==============================] - 90s 543ms/step - loss: 0.0992 - accuracy: 0.9612 - precision: 0.9877 - recall: 0.9599 - val_loss: 0.3475 - val_accuracy: 0.8558 - val_precision: 0.9688 - val_recall: 0.7949
Epoch 4/50
166/166 [==============================] - 89s 538ms/step - loss: 0.0888 - accuracy: 0.9675 - precision: 0.9894 - recall: 0.9668 - val_loss: 2.2391 - val_accuracy: 0.6394 - val_precision: 0.6341 - val_recall: 1.0000
Epoch 5/50
166/166 [==============================] - 89s 539ms/step - loss: 0.0905 - accurac